# test 관련


In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # 대화형 모드

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("------- GPU Working -------")
    print("[Current GPU]: " + str(torch.cuda.get_device_name(0)))
else:
    print("------- CPU Working -------")
print('[PyTorch version]: ', torch.__version__)

------- GPU Working -------
[Current GPU]: NVIDIA GeForce RTX 3090
[PyTorch version]:  1.11.0


In [3]:
batch_size=1
num_workers=8  #16안좋은듯

In [4]:
class_names = 7

model=torchvision.models.resnet152(pretrained=True)

# model_save_dir='C:\\Users\\yj\\Desktop/Task/Trasnfer_Learning/Resnet152/saved_model/F10/20220520_085918/F10_model.pth'
model_save_dir='C:\\Users\\yj\\Desktop/Task/Trasnfer_Learning/Resnet152/Best_model_state/F11_model.pth'

checkpoint = torch.load(model_save_dir)

num_ftrs = model.fc.in_features #체크포인트 로드하기전에 동일하게(마지막단 노드수 7) 변경
model.fc = torch.nn.Linear(num_ftrs, class_names) #(fc): Linear(in_features=2048, out_features=7, bias=True)
model.to(device)

model.load_state_dict(checkpoint)

model.eval()
print('Model loaded!')

Model loaded!


In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
datasetDirectory_path = 'C:\\Users\\yj\\Desktop\\Task\\Trasnfer_Learning\\dataset_per_sub'#sub별 정서이미지폴더 리스트(폴더명)

# datasetDirectory_path=os.path.join(current_dir, data_dir)
datasetDirectory_list = os.listdir(datasetDirectory_path)
print(len(datasetDirectory_list), datasetDirectory_list) #['F1', ~ 'M37']

74 ['F1', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F2', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F3', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'M1', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M2', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25', 'M26', 'M27', 'M28', 'M29', 'M3', 'M30', 'M31', 'M32', 'M33', 'M34', 'M35', 'M36', 'M37', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']


In [10]:
dataloaders_set = []
dataset_sizes_set = []

for i in range(len(datasetDirectory_list)):
    sub = datasetDirectory_list[i]

    path = {x: os.path.join(datasetDirectory_path, datasetDirectory_list[i], x)  for x in ['train', 'valid']} 

    image_datasets = {x: datasets.ImageFolder(path[x],
                                          data_transforms[x])
                                    for x in ['train', 'valid']}
    # print('image_datasets========== ', image_datasets)
    
    dataloaders = { 'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,
                                             shuffle=True, num_workers=num_workers, pin_memory=True),
                'valid' : torch.utils.data.DataLoader(image_datasets['valid'], batch_size=4,
                                             shuffle=False, num_workers=num_workers, pin_memory=True) }
    # print('dataloaders=========', dataloaders)
    dataloaders_set.append(dataloaders)
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
    dataset_sizes_set.append(dataset_sizes)
    
# print('dataloaders_set', dataloaders_set)
# print()
# print('dataset_sizes', dataset_sizes)
# print('dataset_sizes_set', dataset_sizes_set)
# print(dataloaders_set)
# dataset_sizes
# dataset_sizes_set

In [11]:
datasetDirectory_list.index('F11')

2

In [12]:
import torch.nn.functional as F
from tqdm import tqdm

model.eval()

images_so_far = 0

total_loss = 0
total_len = 0
total_correct = 0

with torch.no_grad():
    for i, (sample, label) in enumerate(dataloaders_set[2]['valid']):
        sample = sample.to(device)
        # print(type(inputs))
        # print(inputs.size()) #torch.Size([4, 3, 224, 224])
        # print(inputs.size()[0]) # 4
        label = label.to(device)
        print('label:', label)
        labels = label.clone().detach()  #labels: tensor([0, 1, 2, 3], device='cuda:0')
        # print('labels:', labels)
        
        outputs = model(sample) # 배치4개씩 [7차원]의 텐서 나옴
        # print('outputs', outputs)
        _, preds = torch.max(outputs, 1) #_수치높은값, pred: 수치높은값의 인덱스 tensor([0, 1, 2, 3] #preds tensor([0, 1, 4, 3], device='cuda:0')
        print('preds:', preds)
        if list(preds) != list(labels):
            for p in range(len(preds)):
                print('Not predictd: pred {} label {}'.format(preds[p], labels[p]))
        
        correct = preds.eq(labels) # pred랑 label이랑 같으면 true반환 #correct tensor([ True,  True, False,  True], device='cuda:0')
           # print('prediect: {}, label: {}'.format(preds, labels))
        
        total_correct += correct.sum().item()  #true(맞힌)개수      
        total_len += len(labels) # labels개수=배치 수
    print('Test accuracy: {:.4f}'.format(total_correct / total_len))      


label: tensor([0, 1, 2, 3], device='cuda:0')
preds: tensor([0, 1, 2, 3], device='cuda:0')
label: tensor([4, 5, 6], device='cuda:0')
preds: tensor([4, 5, 6], device='cuda:0')
Test accuracy: 1.0000


# RoBERTa 테스트 코드

In [ ]:
#20211126_121016

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print('Available device : ', torch.cuda.device_count())
print('Current cuda device :', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))


# Setting parameters
batch_size = 200
num_label = 2
learning_rate = 1e-5
max_grad_norm = 1
# epochs = 2
num_workers = 8
# max_len = 512



import time
start_time = time.time()

from transformers import BertTokenizer, RobertaTokenizer, RobertaForSequenceClassification
import torch

config = 'klue/roberta-large'
tokenizer = BertTokenizer.from_pretrained(config)
model = RobertaForSequenceClassification.from_pretrained(config, num_labels=num_label)
model.to(device)


model_save_dir='/home/yeji/standard/model/chitchat_classification/chitchat_classification/Mix_5sentences3/experiment_dataset2/saved_model/epoch_1.pt'
learning_rate = 1e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
checkpoint = torch.load(model_save_dir)

# 원래 module. 없어야됨
# 근데 생겨서 지움
new_checkpoint = { k.replace('module.','') if 'module.' in k else k:v for k,v in checkpoint['model_state_dict'].items()}

model.load_state_dict(new_checkpoint)#checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# model.to(device)

model.eval()
print('Model loaded!')

# =========================================================================

In [12]:
path='C:\\Users\\yj\\Desktop/새 폴더/Dataset/dataset_per_sub'
subject_list = os.listdir(path)
# subject

In [9]:
test_transform = torchvision.transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

test_loader_set = []

for one_sub in subject:
    # try:
    test_path= 'C:\\Users\\yj\\Desktop/새 폴더/Dataset/dataset_per_sub\\'  +  '{}\\test'.format(one_sub)
    test_dataset = torchvision.datasets.ImageFolder(test_path, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_workers)
    test_loader_set.append(test_loader)
    # except:
    #     pass
    # break

In [10]:
def load_checkpoint(subject):

    class_names = 7
    learning_rate =0.1 #1e-5(X)

    model=torchvision.models.resnet152(pretrained=True)

    num_ftrs = model.fc.in_features 
    model.fc = torch.nn.Linear(num_ftrs, class_names) #(fc): Linear(in_features=2048, out_features=7, bias=True)
    model.to(device)

    model_save_dir='C:\\Users\\yj\\Desktop/Task/Trasnfer_Learning/Resnet152/saved_model/20220602_235407/' + '{}.pt'.format(subject)
    # print(model_save_dir) #C:\Users\yj\Desktop/Task/Trasnfer_Learning/Resnet152/saved_model/20220602_235407/F1.pt
    optimizer = torch.optim.SGD(model.fc.parameters(), lr=learning_rate, momentum=0.9,
                                     weight_decay=0.0001)
    checkpoint = torch.load(model_save_dir)

    model.load_state_dict(checkpoint['model_state_dict'])#checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']

    model.eval()
    # print('Model loaded!')

In [11]:
from openpyxl import Workbook
workbook = Workbook( )#workbook 생성하기(1개의 시트가 생성된 상태)
sheet = workbook.active #현재 workbook의 활성화 된 Sheet 가져오기
sheet.title = "정확도" #해당 sheet의 sheet명 변경하기

def test(subject, test_loader):
    model.eval()

    total_loss = 0
    total_len = 0
    total_correct = 0

    for batch_sample, batch_label in test_loader:

        sample = batch_sample.to(device)
        label = batch_label.to(device)

        with torch.no_grad():
            output = model.forward(sample) 
        # loss = loss_func(output, label)     

        _, output_index = torch.max(output, 1)   
        preds = output_index
        # print('preds:', preds)
        # if list(preds) != list(labels):
        #     for p in range(len(preds)):
        #         print('Not predictd: pred {} label {}'.format(preds[p], labels[p]))        
        correct = preds.eq(label)   #1배치에 맞힌 갯수     
        # print('correct', correct)
        total_correct += correct.sum().item()  #1배치에 맞힌 개수 계속 더함        
        # print('total_correct', total_correct)
        total_len += len(label)      #1배치 갯수 계속 더함
        # print('total_len', total_len)

        test_accuracy = total_correct / total_len #이시점까지(에폴당) 누적된(배치마다 계속 맞힌 갯수 더한) 정확도
        # print('test_accuracy', test_accuracy)
    # print('{}/{}  Test accuracy: {:.4f}'.format(epoch + 1, epochs, test_accuracy)) #(에폭당) 전체데이터셋에 대한 정확도 #마지막 배치(앞에서부터 끝까지 누적됨)정확도랑 같음
    print('{} Test accuracy: {:.4f}'.format(subject, test_accuracy)) 
    # write_ws = write_wb.active #디폴트 시트 선택 #현재 workbook의 활성화 된 Sheet 가져오기
    
    # cell에 직접 데이터 입력하기
    sheet['A1'] = "Subject"
    # sheet['B1'] = ""
    
    sheet.append(['{}'.format(subject),
                     'Test accuracy:',
                     '{:.4f}'.format(test_accuracy)])# row 단위로 데이터 입력하기
    # write_ws.append(['{}'.format(subject),
    #                  'Test accuracy:',
    #                  '{:.4f}'.format(test_accuracy)])# row 단위로 데이터 입력하기

    # write_ws.append(['{} Test accuracy: {:.4f}'.format(subject, test_accuracy)])
    workbook.save('./Test_Accuracy.xlsx')

In [100]:
for one_subject, test_loader in zip(subject_list, test_loader_set):
    # print(one_subject, test_loader) #F1 <torch.utils.data.dataloader.DataLoader object at 0x00000207957697F0>
#     
    load_checkpoint(subject=one_subject)
    test(subject=one_subject, test_loader=test_loader)
    # break

F1 Test accuracy: 1.0000
F10 Test accuracy: 0.8571
F11 Test accuracy: 1.0000
F12 Test accuracy: 1.0000
F13 Test accuracy: 1.0000
F14 Test accuracy: 1.0000
F15 Test accuracy: 1.0000
F16 Test accuracy: 1.0000
F17 Test accuracy: 1.0000
F18 Test accuracy: 1.0000
F19 Test accuracy: 1.0000
F2 Test accuracy: 1.0000
F20 Test accuracy: 1.0000
F21 Test accuracy: 1.0000
F22 Test accuracy: 1.0000
F23 Test accuracy: 1.0000
F24 Test accuracy: 1.0000
F25 Test accuracy: 1.0000
F26 Test accuracy: 1.0000
F27 Test accuracy: 1.0000
F28 Test accuracy: 1.0000
F29 Test accuracy: 1.0000
F3 Test accuracy: 1.0000
F30 Test accuracy: 1.0000
F31 Test accuracy: 1.0000
F32 Test accuracy: 1.0000
F33 Test accuracy: 1.0000
F34 Test accuracy: 1.0000
F35 Test accuracy: 1.0000
F36 Test accuracy: 1.0000
F37 Test accuracy: 1.0000
F4 Test accuracy: 1.0000
F5 Test accuracy: 1.0000
F6 Test accuracy: 0.1429
F7 Test accuracy: 1.0000
F8 Test accuracy: 1.0000
F9 Test accuracy: 1.0000
M1 Test accuracy: 1.0000
M10 Test accuracy: 1.0

In [ ]:
def load_checkpoint(final_save_dir): 
    
    if os.path.exists(final_save_dir):        

        checkpoint = torch.load(final_save_dir)
        
        epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        # print("=> loading checkpoint")      
    
    else:
        return None